In [ ]:
%%writefile frontend.py

import streamlit as st
import base64
from pydub import AudioSegment
import pandas as pd
import requests

st.set_page_config(page_title="VoiceCrux", page_icon="🎙️")

hide_streamlit_style = """
    <style>
    header {visibility: hidden;}
    </style>
"""
st.markdown(hide_streamlit_style, unsafe_allow_html=True)

st.title("VoiceCrux")
st.caption("_:blue[AI-powered meeting tool that transcribes recordings with speaker labels and provides smart, concise summaries]_")

input = st.radio("Select Input Type",["Record Audio", "Upload Audio"], index=None)

if input == "Upload Audio":
    st.subheader("Upload Audio")

    uploaded_file = st.file_uploader("Upload an audio file", type=["3gp", "ogg", "opus", "webm", "mp3", "wav", "flac", "aac", "m4a", "wma"])
    if uploaded_file is not None:
        st.audio(uploaded_file)
    number = st.number_input("Enter number of speakers",1,99,placeholder="No of speakers?")
    state = st.button("Process")

    if uploaded_file is not None and number is not None and state:
        audio = AudioSegment.from_file(uploaded_file)
        wav_data = audio.export(format="wav").read()
        base64_audio = base64.b64encode(wav_data).decode()

        response=[]
        with st.spinner():
            response = requests.post("BACKEND_URL", json={"audio": base64_audio, "number": number})

        if response.status_code == 201:
            data = response.json()

            segments = data["d_data"]["segments"]
            start_times = [segment["start"] for segment in segments]
            end_times = [segment["end"] for segment in segments]
            texts = [segment["text"] for segment in segments]
            speakers = [segment["speaker"] for segment in segments]

            df = pd.DataFrame({
                "start_time": start_times,
                "end_time": end_times,
                "text": texts,
                "speaker": speakers
            })

            st.dataframe(df,use_container_width=True)

            st.subheader("Summary")

            for key in data["summary"].keys():
                st.markdown(f"**{key}**:")

                justified_text = f"""
                <div style="text-align: justify;">
                    {data["summary"][key]}
                </div>
                """

                st.markdown(justified_text, unsafe_allow_html=True)

                st.markdown("\n")
        else:
            st.warning(f"Error")

if input == "Record Audio":
    st.subheader("Record Audio")

    audio_value = st.audio_input("Record your audio")
    number = st.number_input("Enter number of speakers",1,99,placeholder="No of speakers?")
    state = st.button("Process")

    if audio_value is not None and number is not None and state:
        base64_audio = base64.b64encode(audio_value.read()).decode()

        response=[]
        with st.spinner():
            response = requests.post("BACKEND_URL", json={"audio": base64_audio, "number": number})

        if response.status_code == 201:
            data = response.json()

            segments = data["d_data"]["segments"]
            start_times = [segment["start"] for segment in segments]
            end_times = [segment["end"] for segment in segments]
            texts = [segment["text"] for segment in segments]
            speakers = [segment["speaker"] for segment in segments]

            df = pd.DataFrame({
                "start_time": start_times,
                "end_time": end_times,
                "text": texts,
                "speaker": speakers
            })

            st.dataframe(df,use_container_width=True)

            st.subheader("Summary")

            for key in data["summary"].keys():
                st.markdown(f"**{key}**:")

                justified_text = f"""
                <div style="text-align: justify;">
                    {data["summary"][key]}
                </div>
                """
                st.markdown(justified_text, unsafe_allow_html=True)

                st.markdown("\n")
        else:
            st.warning(f"Error")

In [ ]:
!pip install pyngrok streamlit pydub

In [3]:
from pyngrok import ngrok

In [7]:
!streamlit run frontend.py &>/dev/null &

In [ ]:
port_no = 8501
ngrok.set_auth_token("NGROK_AUTH_TOKEN")
public_url = "NGROK_PUBLIC_URL"
ngrok_tunnel = ngrok.connect(port_no, bind_tls=True, hostname="NGROK_HOSTNAME")
print(f"Ngrok tunnel: {ngrok_tunnel.public_url}")